In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Web Scraping

In [134]:
# Importando bibliotecas:
import pandas as pd
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

# Declarando variável cards:
cards = []

## Obtendo o HTML e o total de páginas:
url = 'https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/aluguel/apartamentos?o=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
try:
    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    
except HTTPError as e:
    print(e.status, e.reason)
    
except URLError as e:
    print(e.reason)

# Criando o objeto soup:
soup = BeautifulSoup(html, 'html.parser')

# Número de páginas: *dependendo do site pode ser atomatizado ou coletado manualmente como nesse caso
pages = int(29)

## Iterando por todas as páginas do site:
for i in range(pages):
    ## Obtendo o HTML
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    req = Request('https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/aluguel/apartamentos?o=' + str(i + 1), headers=headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    # Obtendo as TAGs de interesse
    anuncios = soup.find('div', {"id": "column-main-content"}).findAll('div', class_="fnmrjs-2 jiSLYe")
    # Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}
        
        # Nome do anúncio
        nome = anuncio.find("h2").getText()
        card['NOME'] = nome
        
        # Valor
        span_valor = anuncio.find("span", class_="sc-ifAKCX eoKYee")
        valor = span_valor.get_text() if span_valor else ''
        card["VALOR"] = valor

        # Detalhes
        span_detalhe = anuncio.find("span", class_="sc-1j5op1p-0 lnqdIU sc-ifAKCX eLPYJb")
        detalhe = span_detalhe.get_text().split('|') if span_detalhe else ''
        card['DETALHES'] = detalhe

        # Endereço
        endereco = anuncio.find('span', class_="sc-7l84qu-1 ciykCV sc-ifAKCX hGjdHN").getText().split(', ')[1]
        card['ENDEREÇO'] = endereco

        # Adicionando resultado a lista cards
        cards.append(card)

# Criando um DataFrame com os resultados
df = pd.DataFrame(cards)
df

,NOME,VALOR,DETALHES,ENDEREÇO
0,Alugo apartamento em condomínio Portamaris com...,R$ 4.000,"[3 quartos , 79m² , 2 vagas]",Ponta Negra
1,Apto tipo suite e apto de 01 e 02 quartos em c...,R$ 380,[0m²],Cidade Alta
2,Apartamento alto padrão,R$ 2.500,"[3 quartos , 118m² , Condomínio: R$ 0 , 2 v...",Candelária
3,Lindo apto locação anual,R$ 1.750,"[1 quarto , 43m² , 1 vaga]",Capim Macio
4,"Apartamento com 2 dormitórios para alugar, 50 ...",R$ 2.100,"[2 quartos , 50m²]",Petrópolis
...,...,...,...,...
1438,Kitnet ponta negra,,"[1 quarto , 111111m² , Condomínio: R$ 0]",Ponta Negra
1439,Alugo no Sun Towers - Cidade Satélite,R$ 950,"[2 quartos , 55m² , Condomínio: R$ 350 , 1 ...",Pitimbu
1440,Apartamento Cidade Alta,R$ 650,"[1 quarto , 32m² , 1 vaga]",Ribeira
1441,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio


# Data Cleaning (dúvidas nos titulos)

In [170]:
print(pd.__version__)

1.1.0


In [135]:
import re

## Separação da coluna DETALHES em colunas QUARTOS, ÁREA, CONDOMINIO E VAGAS

* Cada item da coluna DETALHES é uma ***lista***.
* O principal problema está em transformar cada componente da lista em um ***número inteiro***, o que na minha tentativa só resulta em ***listas de um só valor em string***.

In [148]:
df

,NOME,VALOR,DETALHES,ENDEREÇO
0,Alugo apartamento em condomínio Portamaris com...,R$ 4.000,"[3 quartos , 79m² , 2 vagas]",Ponta Negra
1,Apto tipo suite e apto de 01 e 02 quartos em c...,R$ 380,[0m²],Cidade Alta
2,Apartamento alto padrão,R$ 2.500,"[3 quartos , 118m² , Condomínio: R$ 0 , 2 v...",Candelária
3,Lindo apto locação anual,R$ 1.750,"[1 quarto , 43m² , 1 vaga]",Capim Macio
4,"Apartamento com 2 dormitórios para alugar, 50 ...",R$ 2.100,"[2 quartos , 50m²]",Petrópolis
...,...,...,...,...
1438,Kitnet ponta negra,,"[1 quarto , 111111m² , Condomínio: R$ 0]",Ponta Negra
1439,Alugo no Sun Towers - Cidade Satélite,R$ 950,"[2 quartos , 55m² , Condomínio: R$ 350 , 1 ...",Pitimbu
1440,Apartamento Cidade Alta,R$ 650,"[1 quarto , 32m² , 1 vaga]",Ribeira
1441,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio


## Transformar lista em strings (?)

In [153]:
df['DETALHES_'] = df['DETALHES'].apply(lambda x: (''.join(str(x))))

In [154]:
df['DETALHES_'] = df['DETALHES_'].apply(lambda x: x.replace('[', ''))
df['DETALHES_'] = df['DETALHES_'].apply(lambda x: x.replace(']', ''))
df['DETALHES_']

0                      '3 quartos ', ' 79m² ', ' 2 vagas'
1                                                   '0m²'
2       '3 quartos ', ' 118m² ', ' Condomínio: R$ 0 ',...
3                        '1 quarto ', ' 43m² ', ' 1 vaga'
4                                   '2 quartos ', ' 50m²'
                              ...                        
1438       '1 quarto ', ' 111111m² ', ' Condomínio: R$ 0'
1439    '2 quartos ', ' 55m² ', ' Condomínio: R$ 350 '...
1440                     '1 quarto ', ' 32m² ', ' 1 vaga'
1441                                                     
1442         '1 quarto ', ' 34m² ', ' Condomínio: R$ 565'
Name: DETALHES_, Length: 1443, dtype: object

## Em cada componente se aplicam duas funções regex (?)

* o que no final requer transformar cada conteúdo de linha em string (***str(x)***)

In [158]:
# substituindo os detalhes para que as variaveis QUANTITATIVAS a estejam em tipo inteiro ou float (real)
df['VALOR_INT'] = df['VALOR'].apply(lambda x: x.split(' ')[-1].replace('.',''))

df['QUARTOS'] = df['DETALHES_'].apply(lambda x: re.findall('[0-9]+ quartos?', x))
df['QUARTOS'] = df['QUARTOS'].apply(lambda x: re.findall('[0-9]+', str(x)))

df['ÁREA_INT (m²)'] = df['DETALHES_'].apply(lambda x: re.findall('[0-9]+m²', x))
df['ÁREA_INT (m²)'] = df['ÁREA_INT (m²)'].apply(lambda x: re.findall('[0-9]+', str(x)))

df['COND (R$)'] = df['DETALHES_'].apply(lambda x: re.findall('Condomínio: [0-9]+', x))
df['COND (R$)'] = df['COND (R$)'].apply(lambda x: re.findall('[0-9]+', str(x)))

df['VAGAS'] = df['DETALHES_'].apply(lambda x: re.findall('[0-9]+ vagas?', x))
df['VAGAS'] = df['VAGAS'].apply(lambda x: re.findall('[0-9]', str(x)))

In [159]:
df

,NOME,VALOR,DETALHES,ENDEREÇO,DETALHES_,VALOR_INT,QUARTOS,ÁREA_INT (m²),COND (R$),VAGAS
0,Alugo apartamento em condomínio Portamaris com...,R$ 4.000,"[3 quartos , 79m² , 2 vagas]",Ponta Negra,"'3 quartos ', ' 79m² ', ' 2 vagas'",4000,[3],[79],[],[2]
1,Apto tipo suite e apto de 01 e 02 quartos em c...,R$ 380,[0m²],Cidade Alta,'0m²',380,[],[0],[],[]
2,Apartamento alto padrão,R$ 2.500,"[3 quartos , 118m² , Condomínio: R$ 0 , 2 v...",Candelária,"'3 quartos ', ' 118m² ', ' Condomínio: R$ 0 ',...",2500,[3],[118],[],[2]
3,Lindo apto locação anual,R$ 1.750,"[1 quarto , 43m² , 1 vaga]",Capim Macio,"'1 quarto ', ' 43m² ', ' 1 vaga'",1750,[1],[43],[],[1]
4,"Apartamento com 2 dormitórios para alugar, 50 ...",R$ 2.100,"[2 quartos , 50m²]",Petrópolis,"'2 quartos ', ' 50m²'",2100,[2],[50],[],[]
...,...,...,...,...,...,...,...,...,...,...
1438,Kitnet ponta negra,,"[1 quarto , 111111m² , Condomínio: R$ 0]",Ponta Negra,"'1 quarto ', ' 111111m² ', ' Condomínio: R$ 0'",,[1],[111111],[],[]
1439,Alugo no Sun Towers - Cidade Satélite,R$ 950,"[2 quartos , 55m² , Condomínio: R$ 350 , 1 ...",Pitimbu,"'2 quartos ', ' 55m² ', ' Condomínio: R$ 350 '...",950,[2],[55],[],[1]
1440,Apartamento Cidade Alta,R$ 650,"[1 quarto , 32m² , 1 vaga]",Ribeira,"'1 quarto ', ' 32m² ', ' 1 vaga'",650,[1],[32],[],[1]
1441,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio,,800,[],[],[],[]


## O resultado são colunas com listas de string numéricos

* Imagino que existam funções no python ou pandas que simplifiquem esse processo, mas não encontro caso semelhante no stackoverflow ou Google.

In [160]:
df1 = df[['NOME', 'ENDEREÇO', 'QUARTOS', 'ÁREA_INT (m²)', 'VAGAS', 'COND (R$)', 'VALOR_INT']]
df1

,NOME,ENDEREÇO,QUARTOS,ÁREA_INT (m²),VAGAS,COND (R$),VALOR_INT
0,Alugo apartamento em condomínio Portamaris com...,Ponta Negra,[3],[79],[2],[],4000
1,Apto tipo suite e apto de 01 e 02 quartos em c...,Cidade Alta,[],[0],[],[],380
2,Apartamento alto padrão,Candelária,[3],[118],[2],[],2500
3,Lindo apto locação anual,Capim Macio,[1],[43],[1],[],1750
4,"Apartamento com 2 dormitórios para alugar, 50 ...",Petrópolis,[2],[50],[],[],2100
...,...,...,...,...,...,...,...
1438,Kitnet ponta negra,Ponta Negra,[1],[111111],[],[],
1439,Alugo no Sun Towers - Cidade Satélite,Pitimbu,[2],[55],[1],[],950
1440,Apartamento Cidade Alta,Ribeira,[1],[32],[1],[],650
1441,Kitnet UFRN cidade jardim capim macio,Capim Macio,[],[],[],[],800


In [171]:
df = df.astype({"ÁREA_INT(m²)": float, "COND (R$)": float})
df

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'

In [162]:
df1['QUARTOS'] = df1['QUARTOS'].apply(lambda x: int(x))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [133]:
pd.to_numeric(df1['QUARTOS'][1])

ValueError: Unable to parse string "['3']" at position 0

In [22]:
df['VALOR'].apply(lambda x: x.split(' ')[-1].replace('.',''))

0       2100
1        380
2       1050
3        950
4        750
        ... 
1441    1800
1442    1750
1443     800
1444     900
1445    1200
Name: VALOR, Length: 1446, dtype: object

In [23]:
df['VALOR_INT'] = df['VALOR'].apply(lambda x: x.split(' ')[-1].replace('.',''))

In [27]:
df['DETALHES_'] = df['DETALHES'].apply(lambda x: str(', '.join(x)))

In [32]:
df

,NOME,VALOR,DETALHES,ENDEREÇO,VALOR_INT,DETALHES_
0,"Apartamento com 2 dormitórios para alugar, 50 ...",R$ 2.100,"[2 quartos , 50m²]",Petrópolis,2100,"2 quartos , 50m²"
1,Aluga-se,R$ 380,[1 quarto],Pajuçara,380,1 quarto
2,Apartamento em Candelária - armários em todos ...,R$ 1.050,"[4 quartos , 150m² , Condomínio: R$ 1.050 , ...",Candelária,1050,"4 quartos , 150m² , Condomínio: R$ 1.050 , ..."
3,"Apartamento em Potilândia (Lagoa Nova), vizinh...",R$ 950,"[3 quartos , 88m² , Condomínio: R$ 350 , 1 ...",Lagoa Nova,950,"3 quartos , 88m² , Condomínio: R$ 350 , 1 vaga"
4,Apartamento para aluguel tem 40 metros quadrad...,R$ 750,"[1 quarto , 40m²]",Capim Macio,750,"1 quarto , 40m²"
...,...,...,...,...,...,...
1441,"Apartamento com 1 dormitório para alugar, 34 m...",R$ 1.800,"[1 quarto , 34m² , Condomínio: R$ 565]",Nossa Senhora da Apresentação,1800,"1 quarto , 34m² , Condomínio: R$ 565"
1442,Apt. mobiliado no Ed. Studio da Praia c/ 54 m²...,R$ 1.750,"[1 quarto , 54m² , 1 vaga]",Ponta Negra,1750,"1 quarto , 54m² , 1 vaga"
1443,Apartamento para alugar com 3 dormitórios em C...,R$ 800,"[3 quartos , 76m² , Condomínio: R$ 650 , 1 ...",Capim Macio,800,"3 quartos , 76m² , Condomínio: R$ 650 , 1 vaga"
1444,"Apartamento com 2 dormitórios para alugar, 55 ...",R$ 900,"[2 quartos , 55m² , Condomínio: R$ 470 , 1 ...",Tirol,900,"2 quartos , 55m² , Condomínio: R$ 470 , 1 vaga"


In [33]:
df['DETALHES_'] = df['DETALHES_'].apply(lambda x: x.replace(' R$ ', ' '))